# A one period model

Suppose we want to price an European call option on a stock that matures in six months.

## Model setup

We consider two points in time:
- Today: t=0 
- Six months from now: t=1

<b>Definition:</b> (Arbitrage) Arbitrage pricing is pricing in terms of what is already present on the market.

### Bond

<b>Definition:</b> (Bond) A bond is a fixed income investment in which an investor loans money to an entity (typically corporate or governmental) which borrows the funds for a defined period of time at a variable or fixed interest rate.

Let's consider a bond with price process $B$, whose dynamic is given by:

$$\begin{equation}
    B(0) = 1\\
    B(1) = 1+R
\end{equation}$$

where $R$ is the one period interest rate.

In [1]:
class BondOnePeriod:

    def __init__(self, r):
        """
        The bound only depends on the interest rates. 
        When considering x several bonds, we will multiply the bound value B by x.
        """
        self.r = float(r)
        self.initial_value = 1
        self.final_value = self.initial_value*(1+self.r)

In [2]:
# Define the bond
bond = BondOnePeriod(r = 1)

In [3]:
# Compute the bond's value
print("The bond value is %s")%(bond.final_value)

The bond value is 2.0


***
### Stock
<b>Definition:</b> (Stock) A stock is a type of security that signifies ownership in a corporation and represents a claim on part of the corporation's assets and earnings.

Let's consider a stock with price process $S$, whose dynamic is given by:

$$\begin{equation} 
\begin{split}
S(0) &= s\\
S(1) &= 
    \begin{cases}
      s \cdot u & \text{with probability } p\\
      s \cdot d & \text{with probability } 1-p
    \end{cases}
\end{split}
\end{equation}$$

where we assume that $d<u$. 

This can be rewritten as $S(1) = s \cdot Z$, where $Z \sim Bernoulli(p)$

<b>Definition:</b> (Strike price) the price at which a put or call option can be exercised.

In [5]:
import numpy as np

class StockOnePeriod:
    """
    In this class, we only consider (at the moment) European call option.

    Parameters:
    - s: initial value of the stock
    - u, d: (upward factor, downward factor) possible values at time t=1
    - p: probability that the upward factor is activated

    Attributes:
    - initial_value: initial value of the stock at time t = 0
    - final_value: possible values of the stock at time t = 1
    - k: strike price of the option (is defined)
    - derivative: payoffs at time t = 1 (with respect to k)
    """
    def __init__(self, s, u, d, p):
        """
        Set the parameters s, u, p, d and compute the initial and final values.
        """
        # Define the binomial model
        self.u = float(u)
        self.d = float(d)
        self.p = float(p)

        # Compute initial and final values
        self.initial_value = float(s)
        self.final_value = np.asarray([self.u * self.initial_value, \
                                   self.d * self.initial_value])

    def compute_derivative(self, k):
        """
        The derivative is the difference between the possible values of the stock
        at time t = 1 and the strike price.
        Careful: Valid only for European call option
        """
        # Define the strike price at which the option exercices
        self.k = k
        # Compute the derivative
        self.derivative = np.maximum(self.final_value-self.k, 0)
        return(self.derivative)

In [8]:
# Define stock
stock = StockOnePeriod(s = 100, u = 1.2, d = 0.8, p = 0.8)

In [9]:
# Compute stock prices at time t = 1
print("The value of the price process S at time t = 1 is %s")%(stock.final_value)

The value of the price process S at time t = 1 is [120.  80.]


***
### Derivative

<b>Definition:</b> A financial derivative or contingent claim is a random variable $X$ of the form:

$$X = \phi(S(1)) \iff X = \phi(Z)$$

The price at time $t$ of a claim $X$ is denoted by $\Pi(t;X)=\Pi_{X}(t)$.

In [11]:
# Let's compute the derivative of the option on the stock
print("The price of the derivative at time t = 1 on the stock option is %s")%(stock.compute_derivative(k = 110))

The price of the derivative at time t = 1 on the stock option is [10.  0.]


***
## Portfolios, arbitrage, derivatives and completeness

To find the correct price of the derivative we are going to compare the payoff of the derivative
with the payoff of strategies already present in the market. These strategies are described by
portfolios and their value processes.

***

### Porftolio

<b>Definition: (Portfolio)</b> A portfolio $h$ is a vector $(x, y)$, where 

- $x$ = Number of SEK in your account
- $y$ = number of stocks you own

The value of the portfolio is given by the value process C defined by:

$$\boxed{V^{h}(t) = X \cdot B(t) + y \cdot S(t)}$$

<u>Remark:</u> Note that $h \in \mathbb{R}^{2}$, which means that you are allowed to go short and can buy/sell
as much as you want. Specifically:

- If x<0 and y<0: buy + short
- If x>0 and y<0: sell + short
- If x<0 and y>0: buy + long
- If x>0 and y>0: sell + long

In [28]:
import numpy as np

class Portfolio:

    """
    Parameters:
    - stock: (StockOnePeriod) the stock owned in the Portfolio
    - bond: (BondOnePeriod) the bond owned in the Portfolio
    - x: (Float) number of SEK in the account
    - y: (Float) number of owned stock

    Attributes:
    - initial_value: (Float) Value of the Portfolio at time t=O
    - final_value: (Float) Value of the Portfolio at time t=1
    - replicating: (Boolen) Is the Portfolio replicating the stock
    """

    def __init__(self, stock, bond, replicating_portfolio = False, x = 1, y = 1):
        """
        A portfolio contains by definition both a stock and a bond, that should
        defined prior to initializing a Portfolio.

        Regarding (x, y), there are two ways to initialize those values:
        1. Pass them as parameters, therefore indicating how many stocks and bonds are owned.
        2. (x, y) should replicate the claim X on the stock and should satisfy risk neutral validation formula.
        We therefore compute the x and y based on the above mentioned formula.
        """
        # Stock and bond
        self.bond = bond
        self.stock = stock

        # Construct a replicating portfolio based on the stock and the bond
        if(replicating_portfolio):
            self.x = (1/(1+self.bond.r))*\
                        (self.stock.d*self.stock.derivative[0]-self.stock.u*self.stock.derivative[1])/(self.stock.u - self.stock.d)
            self.y = (1/self.stock.initial_value)*(self.stock.derivative[0]-self.stock.derivative[1])/(self.stock.u - self.stock.d)

        # Define manually the portfolio
        else:
            self.x = x
            self.y = y

        # Compute the initial and final value of the portfolio
        self.initial_value = self.x * self.bond.initial_value + self.y * self.stock.initial_value
        self.final_value = self.x * self.bond.final_value + self.y * self.stock.final_value

    def is_replicating(self, k):
        """
        Based on the stock, the bond, x and y, this method checks whether the portfolio replicates
        the claim X. The portfolio does if its final value is equal to the derivative of the stock.
        """
        if(np.all(self.final_value == self.stock.compute_derivative(k))):
            self.replicating = True
        else:
            self.replicating = False
        return(self.replicating)

    def price_option(self, k):
        """
        If the portfolio replicates the claim X, a proposition indicates that the price of the option
        can be computed as discounted expectations of future payoffs.
        Careful: we should use the martingale probabilities Q, rather than objective probabilities P.
        """
        if(self.is_replicating(k)):
            q = ((1+self.bond.r)-self.stock.d)/(self.stock.u-self.stock.d)
            price_option = (1/(1+self.bond.r))*(q*self.stock.derivative[0] + (1-q)*self.stock.derivative[1])

        return(price_option)

In [29]:
# Define the portfolio
bond = BondOnePeriod(r = 0)
stock = StockOnePeriod(s = 100, u = 1.2, d = 0.8, p = 0.8)
portfolio = Portfolio(x = -20, y = 0.25, stock = stock, bond = bond)   

In [30]:
print("The portfolio is defined by (x, y) = %s"%([portfolio.x, portfolio.y]))

The portfolio is defined by (x, y) = [-20, 0.25]


In [31]:
print("The portfolio is replicating: %s"%(portfolio.is_replicating(k = 110)))

The portfolio is replicating: True


***
### Arbitrage

We compare the derivative with the payoff of strategies already present on the market, to have a consistent pricing. Mathematically this requires no arbitrage opportunities exist on the market:

<b>Definition: (Arbitrage portfolio)</b> A portfolio $h$ is said to be an arbitrage portfolio if $h$ is such that
- $V^{h}(0)=0$
- $P(V^{h}(1)\geq 0) = 1$
- $P(V^{h}(1) > 0) > 0$

A market is said to be free of arbitrage if there exist no arbitrage portfolio.

In [32]:
# Compute intial value of the portfolio
print("The initial value of the portfolio is %s")%(portfolio.initial_value)
print("The portfolio is therefore not an arbitrage portfolio.")

The initial value of the portfolio is 5.0
The portfolio is therefore not an arbitrage portfolio.


***
### Completeness

<b>Definition: (Reachable claim)</b> A claim $X$ is said to be reachable if there exists a portfolio $h$ such that:

$$P(V^{h}(1) = X)=1$$

<b>Definition: (Replicating portfolio)</b> The portfolio $h$ is said to be a replicating portfolio for the claim $X$.

<b>Definition: (Complete market)</b> The market is said to be complete if all claims are reachable.

In [33]:
if(portfolio.is_replicating(k = 110)):
    print("The portfolio is replicating the claim X.")

The portfolio is replicating the claim X.


***
If $X$ is replicated by $h$ then we should have $\Pi(t;X) = V^{h}(t)$ for $t = 0, 1$

<b>Proposition:</b> If the claim $X$ is replicated by the portfolio $h$ then all prices except for 

$$\Pi(t;X)=V^{h}(t)$$

will give rise to arbitrage opportunities.

In [34]:
if(np.all(portfolio.final_value == stock.derivative)):
    print("The condition is met for t = 1")

The condition is met for t = 1


In [35]:
print("Therefore, the price at time t=0 of X should be the initial value of the portfolio %s")%portfolio.initial_value

Therefore, the price at time t=0 of X should be the initial value of the portfolio 5.0


***
<b>Proposition:</b> The model is complete if $u > d$.

<b>Proof:</b>

Fix a claim $X = \phi(Z)$. We need to find a portfolio h suck that $V^{h}(1)=\phi(Z)$.
This translates to:

$$\begin{equation} 
\begin{split}
(1+R)x + suy &= \phi(u)\\
(1+R)x + sdy &= \phi(d)
\end{split}
\end{equation}$$
which is a linear systems of equation, yielding the solution:


$$\begin{equation} 
\begin{split}
x &= \frac{1}{1+R} \cdot \frac{u\phi(d)-d\phi(u)}{u-d}\\
y &= \frac{1}{s} \cdot \frac{\phi(u)-\phi(d)}{u-d}
\end{split}
\end{equation}$$

In [36]:
replicating_portfolio = Portfolio(stock, bond, replicating_portfolio = True)

In [37]:
print("The portfolio is defined by (x, y) = %s"%([replicating_portfolio.x, replicating_portfolio.y]))

The portfolio is defined by (x, y) = [20.000000000000004, 0.25000000000000006]


# Risk neutral validation

In a complete model all claims X can be priced according to $$\boxed{\Pi_{X}(0)=V^{h}(0)}$$ where the replicating portfolio is given by:

$$\begin{equation} 
\begin{split}
x &= \frac{1}{1+R} \cdot \frac{u\phi(d)-d\phi(u)}{u-d}\\
y &= \frac{1}{s} \cdot \frac{\phi(u)-\phi(d)}{u-d}
\end{split}
\end{equation}$$

This means that:

$$\begin{equation} 
\begin{split}
\Pi_{X}(0) &= V^{h}(0)\\
&= xB(0)+yS(0) \\
&= x+ys \\
&= \frac{1}{1+R} \cdot (q \cdot \phi(u) + (1-q) \cdot \phi(d))
\end{split}
\end{equation}$$

where $q = \frac{(1+R) - d}{u-d}$.

<u>Remark:</u> Prices can be computed as discounted expectations of future payoffs, but you should use the martingale  probabilities Q, rather than objective probabilities P.

In [38]:
# Compute the price of the option that satisfies risk-neutral validation equation
print("The price of the option is %s")%(portfolio.price_option(k = 110))

The price of the option is 5.0


## Arbitrage revisited

We have been careful to check that we do not introduce arbitrage opportunities when pricing financial derivatives, but we have not checked that there were no arbitrage opportunities to begin with! Rest assured, under mild conditions the model is free of arbitrage.

<b> Proposition: </b> The model is free of arbitrage if and only if $d < 1+R < u$

The interpretation of the condition d < (1 + R) < u is the return on the stock can not dominate the return on the bond or vice versa.

# Questions

- Are the conditions about an arbitrage portfolio are mild? (2 out of 3 for instance)


# To do

- Proofs of proposition 1 and 2
- Find the definitions of an European call option
- Checkthe course on martingales
